In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

О каждой из них мы знаем:

    сколько детей она родила (признак ceb)
    возраст (age)
    длительность получения образования (educ)
    религиозная принадлежность (religion)
    идеальное, по её мнению, количество детей в семье (idlnchld)
    была ли она когда-нибудь замужем (evermarr)
    возраст первого замужества (agefm)
    длительность получения образования мужем (heduc)
    знает ли она о методах контрацепции (knowmeth)
    использует ли она методы контрацепции (usemeth)
    живёт ли она в городе (urban)
    есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [4]:
botswana = pd.read_csv('botswana.tsv', delimiter='\t')
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [5]:
botswana.religion.nunique()

4

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [6]:
botswana.dropna().shape

(1834, 15)

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо". 

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

    - создать новый бинарный признак x2 = 1, если x1='не применимо', иначе 0;
    - заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

    - Создайте признак nevermarr, равный единице там, где в agefm пропуски.
    - Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
    - Замените NaN в признаке agefm на c_agefm = 0.
    - У объектов, где nevermarr = 1, замените NaN в признаке heduc на c_heduc1 = −1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [7]:
botswana['nevermarr'] = botswana['evermarr'].map({0:1, 1:0})
botswana.drop('evermarr', axis=1, inplace=True)
botswana['agefm'].fillna(0, inplace=True)
botswana['heduc'] = botswana.apply(
    lambda df: -1 if df['nevermarr'] else df['heduc'],
    axis=1
)

In [8]:
sum(botswana['heduc'].isna())

123

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (c_idlnchld = −1, c_heduc2 = −2 (значение -1 мы уже использовали), c_usemeth = −1.

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [10]:
botswana['idlnchld_noans'] = 0
botswana.loc[botswana.idlnchld.isnull(), 'idlnchld_noans'] = 1

botswana['heduc_noans'] = 0
botswana.loc[botswana.heduc.isnull(), 'heduc_noans'] = 1

botswana['usemeth_noans'] = 0
botswana.loc[botswana.usemeth.isnull(), 'usemeth_noans'] = 1

In [11]:
values = {'idlnchld': -1, 'heduc': -2, 'usemeth': -1}
botswana.fillna(value=values, inplace=True)

In [12]:
botswana.dropna(inplace=True)

In [13]:
botswana.size

78264

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2? Округлите до трёх знаков после десятичной точки.

In [14]:
query = 'ceb ~ ' + ' + '.join(botswana.columns.to_list()[1:])
query

'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

In [15]:
m1 = smf.ols(query, 
             data=botswana)
fitted_1 = m1.fit()
print(fitted_1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:06:55   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

3.

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1. 

In [16]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_1.resid, fitted_1.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [17]:
fitted_2 = m1.fit(cov_type='HC1')
print(fitted_2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:06:55   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [18]:
query_2 = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

In [19]:
m2 = smf.ols(query_2, 
             data=botswana)
fitted_3 = m2.fit()
print(fitted_3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:06:55   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [20]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_3.resid, fitted_3.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [21]:
fitted_4 = m2.fit(cov_type='HC1')
print(fitted_4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:06:55   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [22]:
print('F=%f, p=%f, k1=%f' % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением c_usemeth = −1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10^(−8), нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans. 

In [23]:
query_3 = 'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans'

In [24]:
m3 = smf.ols(query_3, 
             data=botswana)
fitted_5 = m3.fit()
print(fitted_5.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:06:56   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.

In [27]:
print('F=%f, p=%s, k1=%f' % m2.fit().compare_f_test(m3.fit()))

F=92.890582, p=3.155200948040263e-40, k1=2.000000


Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [26]:
print(fitted_4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:06:56   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.